In [40]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn import tree
from sklearn.tree import _tree
from operator import itemgetter
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.naive_bayes import GaussianNB
import pickle

from google.cloud import storage
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file("../../msds434-whoop-app-44384939c1f4.json")

In [2]:
project_id = 'msds434-whoop-app'
client = bigquery.Client(credentials=credentials, project=project_id)
query = client.query("select * from `msds434-whoop-app.whoopdataset.whoopmerge`")
df = query.to_dataframe()

In [3]:
# flags for key missing data
df['na_acr'] = df['acute_chronic_strain'].isna() + 0
df['na_workout'] = df['y_workout_strain'].isna() + 0

# imputation
workout_cols = ['acute_chronic_strain', 'workout_strain', 'workout_average_heart_rate','workout_max_heart_rate','workout_kilojoule','zone_one','zone_two','zone_thee','zone_four','zone_five',
					 'y_workout_strain', 'y_workout_average_heart_rate','y_workout_max_heart_rate','y_workout_kilojoule','y_zone_one','y_zone_two','y_zone_thee','y_zone_four','y_zone_five']

for i in df.columns:
	if df[i].dtype == 'int' or df[i].dtype == 'float':
		if i in workout_cols:
			df[i] = df[i].fillna(df[i].min())
		elif i.startswith("y_"):
			df[i] = df[i].fillna(df.groupby('day_of_week')[i].transform('median'))
		elif i.startswith("w_"):
			df[i] = df[i].fillna(df.groupby('day_of_week')[i].transform('median'))
		else:
			pass

In [4]:
same_day_sleep_cols = ['sleep_start_time','light_sleep_time', 'slow_wave_sleep_time','rem_sleep_time','sleep_cycle_count','disturbance_count','respiratory_rate']
yesterday_sleep_cols = ['y_total_sleep_time', 'y_light_sleep_time', 'y_slow_wave_sleep_time','y_rem_sleep_time','y_sleep_cycle_count','y_disturbance_count','y_respiratory_rate', 'y_sleep_performance_perc', 'y_sleep_consistency_perc','y_sleep_efficiency_perc']
yesterday_strain_cols = ['y_kilojoule','y_strain', 'y_avg_heart_rate','y_max_heart_rate']
yesterday_workout_cols = ['y_workout_start_time','y_workout_max_heart_rate', 'y_workout_max_heart_rate','y_workout_kilojoule','y_zone_one','y_zone_two','y_zone_thee','y_zone_four','y_zone_five']
weekly_avgs = ['acute_chronic_strain', 'w_strain','w_sleep_start_time_sd','w_slow_wave_sleep_time','w_light_sleep_time','w_rem_sleep_time','w_recovery_score','w_hrv_milli','w_resting_heart_rate']
df['recovery_score_bin'] = pd.cut(df['recovery_score'], bins=[-float('inf'), 33, 67, float('inf')], labels=['red', 'yellow', 'green'])

In [5]:
# HELPER FUNCTIONS
def getTreeVars( TREE, varNames ) :
   tree_ = TREE.tree_
   varName = [ varNames[i] if i != _tree.TREE_UNDEFINED else "undefined!" for i in tree_.feature ]

   nameSet = set()
   for i in tree_.feature :
       if i != _tree.TREE_UNDEFINED :
           nameSet.add( i )
   nameList = list( nameSet )
   parameter_list = list()
   for i in nameList :
       parameter_list.append( varNames[i] )
   return parameter_list

def getEnsembleTreeVars( ENSTREE, varNames ) :
   importance = ENSTREE.feature_importances_
   index = np.argsort(importance)
   theList = []
   for i in index :
       imp_val = importance[i]
       if imp_val > 0.01 :
           v = int( imp_val / np.max( ENSTREE.feature_importances_ ) * 100 )
           theList.append( ( varNames[i], v ) )
   theList = sorted(theList,key=itemgetter(1),reverse=True)
   return theList

## Model 1: predict next day's recovery

In [84]:
other_cols = ['recovery_score', 'week_of_year','day_of_week','na_acr','na_workout', 'sleep_start_time']
feats = list(set(other_cols + same_day_sleep_cols + yesterday_sleep_cols + yesterday_strain_cols + yesterday_workout_cols + weekly_avgs))
df_recovery_model = df[feats]

X = df_recovery_model.drop('recovery_score',axis = 1)
Y = df_recovery_model['recovery_score']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.8, test_size=0.2, random_state=2)

In [38]:
m01_recovery_tree = tree.DecisionTreeRegressor(max_depth=4)
m01_recovery_tree = m01_recovery_tree.fit(X_train, Y_train)

Y_pred_train = m01_recovery_tree.predict(X_train)
Y_pred_test = m01_recovery_tree.predict(X_test)

rmse_train = math.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
rmse_test = math.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))

print(rmse_train)
print(rmse_test)
getTreeVars(m01_recovery_tree, X_train.columns.values)

7.277269795500423
7.1816578703518195


['w_hrv_milli',
 'y_disturbance_count',
 'w_resting_heart_rate',
 'y_total_sleep_time',
 'sleep_start_time',
 'w_recovery_score',
 'acute_chronic_strain']

In [66]:
m01_recovery_gb = GradientBoostingRegressor(random_state=5,n_iter_no_change = 10, n_estimators=100, learning_rate = 0.1,max_depth = 10, min_samples_leaf=5)
m01_recovery_gb = m01_recovery_gb.fit(X_train, Y_train)

Y_pred_train = m01_recovery_gb.predict(X_train)
Y_pred_test = m01_recovery_gb.predict(X_test)

rmse_train = math.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
rmse_test = math.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))

print(rmse_train)
print(rmse_test)
getEnsembleTreeVars(m01_recovery_gb, X_train.columns.values)

4.440838855377452
7.584322528245038


[('w_recovery_score', 100),
 ('w_hrv_milli', 3),
 ('w_light_sleep_time', 2),
 ('acute_chronic_strain', 1),
 ('respiratory_rate', 1),
 ('w_resting_heart_rate', 1)]

In [73]:
print(min(Y_pred_test))
print(max(Y_pred_test))
print(Y_pred_test.mean())

26.875272793572854
95.06128382849975
65.52469636091467


In [85]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline

m01_recovery_lm = Ridge()
# m01_recovery_lm = make_pipeline(StandardScaler(with_mean=False), Ridge())
m01_recovery_lm = m01_recovery_lm.fit(X_train, Y_train)

Y_pred_train = m01_recovery_lm.predict(X_train)
Y_pred_test = m01_recovery_lm.predict(X_test)

rmse_train = math.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
rmse_test = math.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))

print(rmse_train)
print(rmse_test)

7.802213289319231
6.860056180219036


c:\Users\BrettMele\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=8.31014e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


In [89]:
X_train.columns

Index(['y_sleep_performance_perc', 'na_workout', 'w_slow_wave_sleep_time',
       'y_max_heart_rate', 'y_slow_wave_sleep_time', 'y_zone_thee',
       'y_zone_two', 'y_kilojoule', 'y_respiratory_rate',
       'slow_wave_sleep_time', 'y_zone_four', 'w_rem_sleep_time',
       'y_light_sleep_time', 'disturbance_count', 'y_zone_one',
       'y_total_sleep_time', 'na_acr', 'day_of_week', 'sleep_start_time',
       'y_sleep_consistency_perc', 'w_recovery_score',
       'y_workout_max_heart_rate', 'y_sleep_efficiency_perc', 'y_zone_five',
       'w_sleep_start_time_sd', 'y_workout_kilojoule', 'y_rem_sleep_time',
       'light_sleep_time', 'acute_chronic_strain', 'respiratory_rate',
       'y_workout_start_time', 'w_light_sleep_time', 'week_of_year',
       'w_hrv_milli', 'rem_sleep_time', 'y_disturbance_count',
       'y_avg_heart_rate', 'w_strain', 'y_strain', 'sleep_cycle_count',
       'y_sleep_cycle_count', 'w_resting_heart_rate'],
      dtype='object')

## Model 2: workout performance

In [27]:
other_cols = ['workout_strain','recovery_score','hrv_milli', 'day_of_week', 'na_workout']
feats =  list(set(other_cols + same_day_sleep_cols + yesterday_sleep_cols + yesterday_strain_cols + yesterday_workout_cols + weekly_avgs))
df_workout_model = df[feats]
df_workout_model.loc[df_workout_model['na_workout'] == 0]

X = df_workout_model.drop(columns=['workout_strain','na_workout'],axis = 1)
Y = df_workout_model['workout_strain']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.8, test_size=0.2, random_state=2)

In [29]:
m01_workout_tree = tree.DecisionTreeRegressor(max_depth=2)
m01_workout_tree = m01_workout_tree.fit(X_train, Y_train)

Y_pred_train = m01_workout_tree.predict(X_train)
Y_pred_test = m01_workout_tree.predict(X_test)

rmse_train = math.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
rmse_test = math.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))

print(rmse_train)
print(rmse_test)
getTreeVars(m01_workout_tree, X_train.columns.values)

3.3810826544467294
3.516980816049317


['hrv_milli', 'y_strain', 'y_workout_kilojoule']

In [30]:
m01_workout_gb = GradientBoostingRegressor(random_state=5,n_iter_no_change = 3, n_estimators=500, learning_rate = 0.1,max_depth = 2, min_samples_leaf=3)
m01_workout_gb = m01_workout_gb.fit(X_train, Y_train)

Y_pred_train = m01_workout_gb.predict(X_train)
Y_pred_test = m01_workout_gb.predict(X_test)

rmse_train = math.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
rmse_test = math.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))

print(rmse_train)
print(rmse_test)
getEnsembleTreeVars(m01_workout_gb, X_train.columns.values)

3.2057813915115676
3.4071491744316496


[('y_strain', 100),
 ('y_workout_kilojoule', 72),
 ('y_zone_four', 43),
 ('y_workout_start_time', 20),
 ('y_workout_max_heart_rate', 20),
 ('hrv_milli', 13),
 ('respiratory_rate', 8),
 ('y_zone_one', 5),
 ('acute_chronic_strain', 4),
 ('y_max_heart_rate', 4)]

In [31]:
m01_workout_lm = Ridge()
m01_workout_lm = m01_workout_lm.fit(X_train, Y_train)

Y_pred_train = m01_workout_lm.predict(X_train)
Y_pred_test = m01_workout_lm.predict(X_test)

rmse_train = math.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
rmse_test = math.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))

print(rmse_train)
print(rmse_test)

3.415197220279388
3.6966508027693816


In [90]:
m01_workout_lm.feature_names_in_

array(['y_sleep_performance_perc', 'w_slow_wave_sleep_time',
       'y_max_heart_rate', 'y_slow_wave_sleep_time', 'y_zone_thee',
       'y_zone_two', 'y_kilojoule', 'y_respiratory_rate',
       'slow_wave_sleep_time', 'y_zone_four', 'w_rem_sleep_time',
       'y_light_sleep_time', 'disturbance_count', 'y_zone_one',
       'y_total_sleep_time', 'day_of_week', 'sleep_start_time',
       'y_sleep_consistency_perc', 'w_recovery_score',
       'y_workout_max_heart_rate', 'y_sleep_efficiency_perc',
       'y_zone_five', 'w_sleep_start_time_sd', 'y_workout_kilojoule',
       'y_rem_sleep_time', 'light_sleep_time', 'hrv_milli',
       'acute_chronic_strain', 'respiratory_rate', 'recovery_score',
       'y_workout_start_time', 'w_light_sleep_time', 'w_hrv_milli',
       'rem_sleep_time', 'y_disturbance_count', 'y_avg_heart_rate',
       'w_strain', 'y_strain', 'sleep_cycle_count', 'y_sleep_cycle_count',
       'w_resting_heart_rate'], dtype=object)

## Model 3: HRV

In [19]:
other_cols = ['hrv_milli', 'week_of_year','day_of_week','na_acr','na_workout', 'sleep_start_time']
feats = list(set(other_cols + yesterday_sleep_cols + yesterday_strain_cols + yesterday_workout_cols + weekly_avgs))
df_hrv_model = df[feats]

X = df_hrv_model.drop('hrv_milli',axis = 1)
Y = df_hrv_model['hrv_milli']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.8, test_size=0.2, random_state=2)

In [20]:
m01_hrv_tree = tree.DecisionTreeRegressor(max_depth=2)
m01_hrv_tree = m01_hrv_tree.fit(X_train, Y_train)

Y_pred_train = m01_hrv_tree.predict(X_train)
Y_pred_test = m01_hrv_tree.predict(X_test)

rmse_train = math.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
rmse_test = math.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))

print(rmse_train)
print(rmse_test)
getTreeVars(m01_hrv_tree, X_train.columns.values)

19.00053228893981
18.571146346991487


['w_recovery_score']

In [21]:
m01_hrv_gb = GradientBoostingRegressor(random_state=5,n_iter_no_change = 10, n_estimators=500, learning_rate = 0.1,max_depth = 2, min_samples_leaf=3)
m01_hrv_gb = m01_hrv_gb.fit(X_train, Y_train)

Y_pred_train = m01_hrv_gb.predict(X_train)
Y_pred_test = m01_hrv_gb.predict(X_test)

rmse_train = math.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
rmse_test = math.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))

print(rmse_train)
print(rmse_test)
getEnsembleTreeVars(m01_hrv_gb, X_train.columns.values)

15.098555392446734
15.92125753217055


[('w_recovery_score', 100),
 ('y_kilojoule', 12),
 ('y_respiratory_rate', 11),
 ('y_max_heart_rate', 10),
 ('w_hrv_milli', 6),
 ('w_rem_sleep_time', 5),
 ('y_avg_heart_rate', 5),
 ('w_light_sleep_time', 5),
 ('y_disturbance_count', 2),
 ('y_sleep_consistency_perc', 2),
 ('na_acr', 2),
 ('y_workout_max_heart_rate', 2),
 ('y_total_sleep_time', 2),
 ('y_zone_one', 1)]

In [22]:
m01_hrv_lm = Ridge()
m01_hrv_lm = m01_workout_lm.fit(X_train, Y_train)

Y_pred_train = m01_hrv_lm.predict(X_train)
Y_pred_test = m01_hrv_lm.predict(X_test)

rmse_train = math.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
rmse_test = math.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))

print(rmse_train)
print(rmse_test)

16.555736033508587
15.623343282674554


c:\Users\BrettMele\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=9.1822e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


## Pickle them

In [86]:
pickle.dump(m01_hrv_lm, open("hrv_model.pkl", "wb"))
pickle.dump(m01_recovery_lm, open("recovery_model.pkl", "wb"))
pickle.dump(m01_workout_lm, open("strain_model.pkl", "wb"))

In [87]:
client = storage.Client(project='msds434-whoop-app')
bucket = client.get_bucket("whoopdata")

blob = storage.blob.Blob.from_string("gs://hrvmodel/model.pkl", client)
blob.upload_from_filename("hrv_model.pkl")

blob = storage.blob.Blob.from_string("gs://recoverymodel/model.pkl", client)
blob.upload_from_filename("recovery_model.pkl")

blob = storage.blob.Blob.from_string("gs://strainmodel/model.pkl", client)
blob.upload_from_filename("strain_model.pkl")

c:\Users\BrettMele\anaconda3\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
